In [190]:
DOWNLOAD_LATEST = False
import os
if DOWNLOAD_LATEST:
    env_vars = open(".env.development").read()
    DB_CONNECTION_STRING = env_vars.split("DB_CONNECTION_STRING = ")[1].split("?authSource")[0]
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection users --out data/Curio/users.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection posts --out data/Curio/posts.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection groups --out data/Curio/groups.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection reactions --out data/Curio/reactions.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection echopostproposals --out data/Curio/echopostproposals.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection viewtimestamps --out data/Curio/viewtimestamps.json")

In [191]:
import jsonlines
import pandas as pd
from superdebug import debug
from collections import defaultdict
curio_data_path = "data/Curio/"
users = jsonlines.Reader(open(curio_data_path + "users.json"))
posts = jsonlines.Reader(open(curio_data_path + "posts.json"))
groups = jsonlines.Reader(open(curio_data_path + "groups.json"))
reactions = jsonlines.Reader(open(curio_data_path + "reactions.json"))
echopostproposals = jsonlines.Reader(open(curio_data_path + "echopostproposals.json"))
viewtimestamps = jsonlines.Reader(open(curio_data_path + "viewtimestamps.json"))

##### Record necessary information: `group_id_name_map, group_id_members_id_map, user_id_name_map, user_id_group_id_viewtime_map`

In [192]:
user_id_name_map = {}
for user in users:
    user_id = user['_id']["$oid"]
    user_name = user["username"]
    user_id_name_map[user_id] = user_name
    
group_id_members_id_map = {}
group_id_members_name_map = {}
group_name_member_num_map = {}
group_id_name_map = {}
for group in groups:
    group_id = group['_id']["$oid"]
    group_name = group["name"].replace(" ", "|")
    group_id_name_map[group_id] = group_name
    members = [_["$oid"] for _ in group['members']]
    group_id_members_id_map[group_id] = members
    group_id_members_name_map[group_name] = [user_id_name_map[user_id] for user_id in members if user_id in user_id_name_map]
    if "TEST" not in group_name:
        group_name_member_num_map[group_name] = len(members)
debug(group_member_num = group_name_member_num_map)
print("group_id_members_name_map:", group_id_members_name_map)
    
user_id_group_id_viewtime_map = defaultdict(dict)
for viewtimestamp in viewtimestamps:
    if viewtimestamp["user"] is None: continue
    user_id = viewtimestamp["user"]["$oid"]
    if "group" not in viewtimestamp or viewtimestamp["group"] == None:
        group_ids = []
        for group_id in group_id_members_id_map:
            if user_id in group_id_members_id_map[group_id]:
                group_ids.append(group_id)
    else:
        group_ids = [viewtimestamp["group"]["$oid"]]
    dt_obj = pd.to_datetime(viewtimestamp["timestamp"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    for group_id in group_ids:
        if group_id not in user_id_group_id_viewtime_map[user_id] or timestamp > user_id_group_id_viewtime_map[user_id][group_id]:
            user_id_group_id_viewtime_map[user_id][group_id] = timestamp
    

------------------ 2022-12-23 08:02:44 ------------------
DEBUG: 1 vars: ['group_member_num'], at <ipython-input-192-5d1bfe551d68>:20 <module>
0 / 188.  group_member_num dict {...} with 20 keys ['LOLCHI', 'Trivia', 'Team|Llama', 'Team|Mignon', 'The|Positive|Corner', 'Random', 'Food', 'Out|of|Context|Quotes', 'Team|PhThree', 'Stanford|HCI', 'Campus|Events', 'Stanford|Wordle', 'Where|Am|I', 'Bugs', 'Cute|Animals', 'Music', 'Amazing|AI', 'World|Cup|2022', 'Stanford', 'Hot|Takes']
    LOLCHI num val: 26
    Trivia num val: 18
    Team|Llama num val: 7
    Team|Mignon num val: 9
    The|Positive|Corner num val: 18
    Random num val: 18
    Food num val: 24
    Out|of|Context|Quotes num val: 19
    Team|PhThree num val: 7
    Stanford|HCI num val: 24
    Campus|Events num val: 6
    Stanford|Wordle num val: 4
    Where|Am|I num val: 9
    Bugs num val: 12
    Cute|Animals num val: 9
    Music num val: 9
    Amazing|AI num val: 6
    World|Cup|2022 num val: 3
    Stanford num val: 11
    Hot

##### Aggregate information about each post, and compose `posts_df`

In [193]:
import time
from collections import defaultdict, Counter
posts_info_map = defaultdict(dict)
group_name_post_num_map = defaultdict(int)
group_name_first_post_time_map = defaultdict(int)
group_name_active_post_member_map = defaultdict(Counter)
group_name_active_member_map = defaultdict(Counter)
active_member_day_thres = 10000 # TODO:
for post in posts:
    group_id = post["group"]["$oid"]
    group_name = group_id_name_map[group_id]
    if "TEST" in group_name or "test" in group_name:
        continue
    
    group_name_post_num_map[group_name] += 1
    
    post_id = post["_id"]["$oid"]
    post_info = posts_info_map[post_id] # information about the post
    post_info["SUBMISSION_ID"] = post_id
    
    post_info["SUBREDDIT"] = group_name
    post_info["group_id"] = group_id
    title = post["contents"]
    post_info["TITLE"] = title
    
    author_id = post["author"]["$oid"]
    author = user_id_name_map[author_id]
    post_info["author_id"] = author_id
    post_info["AUTHOR"] = author
    post_info["NSFW"] = None
    post_info["LINK"] = ""
    
    dt_obj = pd.to_datetime(post["updatedAt"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    post_info["timestamp"] = timestamp
    if (group_name not in group_name_first_post_time_map) or timestamp < group_name_first_post_time_map[group_name]:
        group_name_first_post_time_map[group_name] = timestamp
    
    if abs(time.time() - timestamp) <= active_member_day_thres * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name][author_id] += 1
        group_name_active_post_member_map[group_name][author_id] += 1
    
    if "#_COMMENTS" not in post_info:
        post_info["#_COMMENTS"] = 0
        
    if "threadParent" in post:
        threadParent_id = post["threadParent"]["$oid"]
        if "#_COMMENTS" not in posts_info_map[threadParent_id]:
            posts_info_map[threadParent_id]["#_COMMENTS"] = 1
        else:
            posts_info_map[threadParent_id]["#_COMMENTS"] += 1

debug(group_active_post_member_num = str({name: len(members) for name, members in group_name_active_post_member_map.items()}))
debug(group_active_post_member_num_2 = str({name: len([member for member in members if members[member] >=2]) for name, members in group_name_active_post_member_map.items()}))

------------------ 2022-12-23 08:02:45 ------------------
DEBUG: 1 vars: ['group_active_post_member_num'], at <ipython-input-193-870b060c80b3>:53 <module>
0 / 189.  group_active_post_member_num str len 348: {'LOLCHI': 11, 'Trivia': 9, 'Random': 7, 'Team|Llama': 5, 'Team|Mignon': 4, 'The|Positive|Corner': 9, 'Food': 6, 'Out|of|Context|Quotes': 9, 'Team|PhThree': 3, 'Campus|Events': 2, 'Where|Am|I': 5, 'Stanford': 2, 'Stanford|Wordle': 1, 'Cute|Animals': 1, 'Music': 4, 'Bugs': 6, 'Stanford|HCI': 1, 'Amazing|AI': 2, 'World|Cup|2022': 3, 'Hot|Takes': 1}
------------------ 2022-12-23 08:02:45 ------------------
------------------ 2022-12-23 08:02:45 ------------------
DEBUG: 1 vars: ['group_active_post_member_num_2'], at <ipython-input-193-870b060c80b3>:54 <module>
0 / 190.  group_active_post_member_num_2 str len 347: {'LOLCHI': 7, 'Trivia': 6, 'Random': 6, 'Team|Llama': 4, 'Team|Mignon': 4, 'The|Positive|Corner': 8, 'Food': 6, 'Out|of|Context|Quotes': 7, 'Team|PhThree': 3, 'Campus|Events':

Analyze statistics

In [194]:
group_name_post_frequency_map = {group_name: (60 * 60 * 24 * group_name_post_num_map[group_name])/(time.time() - group_name_first_post_time_map[group_name]) for group_name in group_name_post_num_map}
print({group_name: f"{group_name_post_frequency_map[group_name]:.4f}" for group_name in group_name_post_frequency_map})
debug(group_posting_frequency_day = {group_name: f"{group_name_post_frequency_map[group_name]:.4f}" for group_name in group_name_post_frequency_map})

group_name_post_frequency_per_person_map = {group_name: group_name_post_frequency_map[group_name]/group_name_member_num_map[group_name] for group_name in group_name_post_frequency_map}
print({group_name: f"{group_name_post_frequency_per_person_map[group_name]:.4f}" for group_name in group_name_post_frequency_per_person_map})
debug(group_posting_frequency_per_person_day = {group_name: f"{group_name_post_frequency_per_person_map[group_name]:.4f}" for group_name in group_name_post_frequency_per_person_map})
posts_df = pd.DataFrame.from_records(list(posts_info_map.values()))

{'LOLCHI': '0.1705', 'Trivia': '0.2683', 'Random': '0.2038', 'Team|Llama': '0.4457', 'Team|Mignon': '0.5684', 'The|Positive|Corner': '0.3451', 'Food': '0.2474', 'Out|of|Context|Quotes': '0.3836', 'Team|PhThree': '0.0943', 'Campus|Events': '0.5048', 'Where|Am|I': '0.4206', 'Stanford': '0.0505', 'Stanford|Wordle': '0.0216', 'Cute|Animals': '0.2081', 'Music': '0.3264', 'Bugs': '0.3835', 'Stanford|HCI': '0.0548', 'Amazing|AI': '0.1668', 'World|Cup|2022': '0.2883', 'Hot|Takes': '0.0949'}
------------------ 2022-12-23 08:02:45 ------------------
DEBUG: 1 vars: ['group_posting_frequency_day'], at <ipython-input-194-43d8415bbca7>:3 <module>
0 / 191.  group_posting_frequency_day dict {...} with 20 keys ['LOLCHI', 'Trivia', 'Random', 'Team|Llama', 'Team|Mignon', 'The|Positive|Corner', 'Food', 'Out|of|Context|Quotes', 'Team|PhThree', 'Campus|Events', 'Where|Am|I', 'Stanford', 'Stanford|Wordle', 'Cute|Animals', 'Music', 'Bugs', 'Stanford|HCI', 'Amazing|AI', 'World|Cup|2022', 'Hot|Takes']
    LOLCH

##### Record users' votes on posts and compose `votes_df`

In [195]:
posts_users_votes = defaultdict(dict)
for post_id, post_info in posts_info_map.items():
    if "SUBMISSION_ID" not in post_info or "SUBREDDIT" not in post_info: continue
    if post_info["SUBREDDIT"] == "" or post_info["SUBREDDIT"] == None: continue
    assert post_id == post_info["SUBMISSION_ID"]
    post_timestamp = post_info["timestamp"]
    group_id = post_info["group_id"]
    author_id = post_info["author_id"]
    group_name = post_info["SUBREDDIT"]
    group_members = group_id_members_id_map[group_id]
    
    # fill in downvotes
    users_votes = posts_users_votes[post_id]
    for member_id in group_members:
        if member_id in user_id_name_map and (group_id not in user_id_group_id_viewtime_map[member_id] or post_timestamp < user_id_group_id_viewtime_map[member_id][group_id]):
            users_votes[member_id] = {"SUBMISSION_ID": post_id,	"SUBREDDIT": "r/" + group_name, "CREATED_TIME": post_timestamp, "USERNAME": user_id_name_map[member_id], "VOTE": "downvote"}
    if author_id in users_votes:
        users_votes[author_id]["VOTE"] = "upvote" # author always upvotes
    else:
        users_votes[author_id] = {"SUBMISSION_ID": post_id,	"SUBREDDIT": "r/" + group_name, "CREATED_TIME": post_timestamp, "USERNAME": user_id_name_map[author_id], "VOTE": "upvote"}

In [196]:
import copy


for reaction in reactions:
    post_id = reaction["post"]["$oid"]
    if post_id not in posts_users_votes: continue
    author_id = reaction["author"]["$oid"]
    if author_id in posts_users_votes[post_id]:
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
    else:
        posts_users_votes[post_id][author_id] = copy.deepcopy(list(posts_users_votes[post_id].values())[0])
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        posts_users_votes[post_id][author_id]["USERNAME"] = user_id_name_map[author_id]
    group_id = posts_info_map[post_id]["group_id"]
    group_name = group_id_name_map[group_id]
    dt_obj = pd.to_datetime(reaction["updatedAt"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    if abs(time.time() - timestamp) <= active_member_day_thres * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name][author_id] += 1

for echopostproposal in echopostproposals:
    post_id = echopostproposal["echoParent"]["$oid"]
    if post_id not in posts_users_votes: continue
    author_id = echopostproposal["echoAuthor"]["$oid"]
    if author_id not in posts_users_votes[post_id]:
        posts_users_votes[post_id][author_id] = copy.deepcopy(list(posts_users_votes[post_id].values())[0])
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        posts_users_votes[post_id][author_id]["USERNAME"] = user_id_name_map[author_id]
    else:
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        
    group_id = posts_info_map[post_id]["group_id"]
    group_name = group_id_name_map[group_id]
    
    post_timestamp = posts_info_map[post_id]["timestamp"]
    if abs(time.time() - post_timestamp) <= active_member_day_thres * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name][author_id] += 1

In [197]:
import json


all_votes = []
group_name_upvote_num_map = defaultdict(int)
for post_id, users_votes in posts_users_votes.items():
    for user_id, vote in users_votes.items():
        all_votes.append(vote)
        if vote["VOTE"] == "upvote":
            group_name_upvote_num_map[vote["SUBREDDIT"][2:]] += 1
# group_name_upvote_frequency_map = {group_name: f"{(time.time() - group_name_first_post_time_map[group_name])/(60 * 60 * 24 * group_name_upvote_num_map[group_name]):.4f}" for group_name in group_name_upvote_num_map}
# debug(group_upvoteing_frequency_day = group_name_upvote_frequency_map)

group_name_upvote_frequency_map = {group_name: (60 * 60 * 24 * group_name_upvote_num_map[group_name])/(time.time() - group_name_first_post_time_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_frequency_map[group_name]:.4f}" for group_name in group_name_upvote_frequency_map})
debug(group_posting_frequency_day = {group_name: f"{group_name_upvote_frequency_map[group_name]:.4f}" for group_name in group_name_upvote_frequency_map})
debug(group_active_member_num_1 = str({name: len(members) for name, members in group_name_active_member_map.items()})) # number of members who have at least posted/reacted once
debug(group_active_member_num_2 = str({name: len([member for member in members if members[member] >=2]) for name, members in group_name_active_member_map.items()}))
group_active_member_2 = {name: [user_id_name_map[member] for member in members if members[member] >=2] for name, members in group_name_active_member_map.items()}
json.dump(group_active_member_2, open("data/Curio/community_curators.json", "w")) # Consider only members who have posted/reacted at least twice as curators
debug(group_active_member_2 = str(group_active_member_2))
votes_df = pd.DataFrame.from_records(all_votes)

{'LOLCHI': '0.4645', 'Trivia': '0.3961', 'Random': '0.4776', 'Team|Llama': '0.5285', 'Team|Mignon': '0.6834', 'The|Positive|Corner': '0.8563', 'Food': '0.6620', 'Out|of|Context|Quotes': '1.1643', 'Team|PhThree': '0.1212', 'Campus|Events': '0.8244', 'Where|Am|I': '0.7065', 'Stanford': '0.0505', 'Stanford|Wordle': '0.0216', 'Cute|Animals': '0.8324', 'Music': '0.6527', 'Bugs': '0.5478', 'Stanford|HCI': '0.0548', 'Amazing|AI': '0.3892', 'World|Cup|2022': '0.2883', 'Hot|Takes': '0.0949'}
------------------ 2022-12-23 08:02:47 ------------------
DEBUG: 1 vars: ['group_posting_frequency_day'], at <ipython-input-197-724a342a08fb>:16 <module>
0 / 193.  group_posting_frequency_day dict {...} with 20 keys ['LOLCHI', 'Trivia', 'Random', 'Team|Llama', 'Team|Mignon', 'The|Positive|Corner', 'Food', 'Out|of|Context|Quotes', 'Team|PhThree', 'Campus|Events', 'Where|Am|I', 'Stanford', 'Stanford|Wordle', 'Cute|Animals', 'Music', 'Bugs', 'Stanford|HCI', 'Amazing|AI', 'World|Cup|2022', 'Hot|Takes']
    LOLC

In [198]:
group_name_active_member_map["Stanford|Wordle"]

Counter({'62b4d327507079b4b1f2d5f2': 1})

In [199]:
group_name_upvote_rate_map = {group_name: (group_name_upvote_num_map[group_name])/(group_name_post_num_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_rate_map[group_name]:.4f}" for group_name in group_name_upvote_rate_map})
group_name_upvote_rate_per_person_map = {group_name: (group_name_upvote_num_map[group_name])/(group_name_post_num_map[group_name] * group_name_member_num_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_rate_per_person_map[group_name]:.4f}" for group_name in group_name_upvote_rate_per_person_map})

{'LOLCHI': '2.7241', 'Trivia': '1.4762', 'Random': '2.3438', 'Team|Llama': '1.1857', 'Team|Mignon': '1.2022', 'The|Positive|Corner': '2.4815', 'Food': '2.6757', 'Out|of|Context|Quotes': '3.0351', 'Team|PhThree': '1.2857', 'Campus|Events': '1.6333', 'Where|Am|I': '1.6800', 'Stanford': '1.0000', 'Stanford|Wordle': '1.0000', 'Cute|Animals': '4.0000', 'Music': '2.0000', 'Bugs': '1.4286', 'Stanford|HCI': '1.0000', 'Amazing|AI': '2.3333', 'World|Cup|2022': '1.0000', 'Hot|Takes': '1.0000'}
{'LOLCHI': '0.1048', 'Trivia': '0.0820', 'Random': '0.1302', 'Team|Llama': '0.1694', 'Team|Mignon': '0.1336', 'The|Positive|Corner': '0.1379', 'Food': '0.1115', 'Out|of|Context|Quotes': '0.1597', 'Team|PhThree': '0.1837', 'Campus|Events': '0.2722', 'Where|Am|I': '0.1867', 'Stanford': '0.0909', 'Stanford|Wordle': '0.2500', 'Cute|Animals': '0.4444', 'Music': '0.2222', 'Bugs': '0.1190', 'Stanford|HCI': '0.0417', 'Amazing|AI': '0.3889', 'World|Cup|2022': '0.3333', 'Hot|Takes': '0.3333'}


In [200]:
posts_df.to_csv(curio_data_path + "post_info.txt", sep = "\t", index = False)
votes_df.to_csv(curio_data_path + "all_votes.txt", sep = "\t", index = False)